### This notebook imports a reference grid, reconstructs it according to a specified plate model and exports the rotated grid

In [1]:
import numpy as np
import pandas as pd
import pygplates as pygp

#### User input: time range, plate model files and desired output file location

In [2]:
start = 10    # start time (in Ma) = youngest reconstruction step (e.g. not t=0)
stop = 540    # stop time (in Ma) = oldest reconstruction step
step = 10     # temporal step size (in Ma)

reference_grid = 'reference_grid/reference_grid.csv'              # reference grid file
static_polygons = 'plate_models/TC16/TC16_static_polygons.gpml'   # static polygon file
rotation_model = 'plate_models/TC16/TC16_rot_model.rot'           # rotation file
anchor_plate = 1                                                  # anchor plate ID (sets the effective reference frame)

rotated_grid = 'rotated_grids/TC16.csv'                           # output file name

#### Import reference grid and build dataframe to append rotated points

In [3]:
df = pd.read_csv(reference_grid, usecols=[0,1], header=0).dropna().reset_index(drop=True)        # ignore empty lines
df['pygp_pts'] = df.apply(lambda row: pygp.PointOnSphere((row.lat, row.lng)), axis=1)            # convert lat/lons to pygplates point format and append as new column
df['plt_id'] = np.nan                     # plate id (to be assigned by partitioning step)
df['maxage'] = np.nan                     # maximum age of the reconstructable point (to be assigned by partitioning step)
df['pygp_feature'] = np.nan               # bundled pygplates data (point, plate id and valid age)
df['plt_id'] = df['plt_id'].astype('Int64')      # this allows this column of NaNs to later be updated with ints

#### Assign plate IDs to each reference grid point

In [4]:
point_features = []
for i in df['pygp_pts']:                  # bundle all the points together into a gplates feature file to expedite partitioning function call
    point_feature = pygp.Feature()
    point_feature.set_geometry(i)
    point_features.append(point_feature)
    
partitioned_features = pygp.partition_into_plates(static_polygons, rotation_model, point_features,
                                                  properties_to_copy = [pygp.PartitionProperty.reconstruction_plate_id,
                                                                        pygp.PartitionProperty.valid_time_period],
                                                  sort_partitioning_plates=pygp.SortPartitioningPlates.by_plate_area)
partitioned_data = []
for i in partitioned_features:            # extract plate ID, maximum age and point location from partitioned features
    point = i.get_geometry()
    plt_id = i.get_reconstruction_plate_id()
    if plt_id == 0: plt_id = np.nan
    begin_time, end_time = i.get_valid_time()
    partitioned_data.append([point, plt_id, begin_time, i])

for i in partitioned_data:                # append these data to our dataframe
    df.loc[df['pygp_pts'] == i[0], ['plt_id', 'maxage', 'pygp_feature']] = i[1], i[2], i[3]
    
    # *** the line above will introduce problems if there are duplicate points (same lat/lon). 
    # if duplicate points are expected use the two lines below instead (this will be slight slower):
    # idx = df[(df.pygp_pts == i[0]) & (pd.isnull(df.pygp_feature))].index[0] 
    # df.loc[idx,['plt_id', 'maxage', 'pygp_feature']] = i[1], i[2], i[3]

#### Reconstruct points through time range and append to dataframe

In [5]:
for time in range(start, stop+step, step):
    reconstruct = df.loc[(df['plt_id'] != np.nan) & (df['maxage'] >= time)].copy()  # for each timestep select subset of reconstructable points
                                        
    reconstructed_geometries = []
    pygp.reconstruct(reconstruct['pygp_feature'], rotation_model, reconstructed_geometries, time, anchor_plate_id = anchor_plate) # reconstruct points
    reconstructed_points = [x.get_reconstructed_geometry().to_lat_lon() for x in reconstructed_geometries]    # extract lat and lon from reconstructed points
    reconstructed_lats = [x[0] for x in reconstructed_points]
    reconstructed_lons = [x[1] for x in reconstructed_points]
    reconstruct[f'lat_{time}'] = reconstructed_lats
    reconstruct[f'lng_{time}'] = reconstructed_lons
    df = pd.concat([df, reconstruct[f'lng_{time}'], reconstruct[f'lat_{time}']], axis=1, join='outer') # append reconstructed lat & lons back to dataframe

#### Tidy dataframe and save

In [6]:
df = df.drop(columns=['pygp_pts', 'plt_id', 'maxage', 'pygp_feature'])   # drop columns that we don't need
df.to_csv(rotated_grid, index=False, na_rep='NA', float_format='%.4f')   # set format of NaNs + precision of floats and save